<a href="https://colab.research.google.com/github/sdx0112/Chinese-Topic-Modelling/blob/main/Aggregate_Topic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use ChatGLM2 to pick the most similar pre-defined for each given topic and each given title by comparing their embeddings.

In [1]:
!git clone https://github.com/THUDM/ChatGLM2-6B

Cloning into 'ChatGLM2-6B'...
remote: Enumerating objects: 205, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 205 (delta 93), reused 67 (delta 66), pack-reused 87
Receiving objects: 100% (205/205), 7.34 MiB | 20.95 MiB/s, done.
Resolving deltas: 100% (107/107), done.


In [2]:
import os
os.chdir('/content/ChatGLM2-6B')

In [3]:
import pandas as pd
import numpy as np
from collections import Counter

In [4]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 79.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.6/416.6 kB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 90.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 132.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.8/288.8 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 8.5 MB/s eta 0:00:0

In [5]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True)
model = AutoModel.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True, device='cuda')
model = model.eval()

A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm2-6b:
- tokenization_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm2-6b:
- configuration_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm2-6b:
- quantization.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm2-6b:
- modeling_chatglm.py
- quantization.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [6]:
import torch
from sklearn.metrics.pairwise import cosine_similarity

## Get embedding from ChatGLM2-6B

In [7]:
def get_glm_embedding(text, device="cuda"):
    inputs = tokenizer([text], return_tensors="pt").to(device)
    resp = model.transformer(**inputs, output_hidden_states=True)
    y = resp.last_hidden_state
    y_mean = torch.mean(y, dim=0, keepdim=True)
    return y_mean.cpu().detach().numpy()

In [8]:
topics_pdf = ['基建', '房地产', '制造业', '科技', '零售消费', '绿色发展', '医药健康', '教育', '交通运输', '出口贸易', '农业农村',
          '文化旅游', '就业', '民生扶贫', '能源', '物资储备', '宏观', '区域发展', '企业发展', '新冠疫情', '地方财政', '税务',
          '国际关系', '金融']

In [9]:
df_para = pd.read_csv('/content/para_topic.csv', encoding='utf_8_sig')

In [11]:
df_title = pd.read_csv('/content/title_topic.csv', encoding='utf_8_sig')

In [12]:
df_para.head()

,Unnamed: 0,ID,TITLE,CONTENT,TOPIC
0,0,m-1608357555-1682084460,会议审议通过了关于强化企业科技创新主体地位的意见关于加强和改进国有经济管理有力支持中国式现代...,中共中央总书记、国家主席、中央军委主席、中央全面深化改革委员会主任习近平4月21日下午主持召...,宏观
1,1,m-1608357555-1682084460,会议审议通过了关于强化企业科技创新主体地位的意见关于加强和改进国有经济管理有力支持中国式现代...,会议审议通过了《关于强化企业科技创新主体地位的意见》、《关于加强和改进国有经济管理有力支持中...,企业发展
2,2,m-1608357555-1682084460,会议审议通过了关于强化企业科技创新主体地位的意见关于加强和改进国有经济管理有力支持中国式现代...,会议指出，强化企业科技创新主体地位，是深化科技体制改革、推动实现高水平科技自立自强的关键举措...,科技
3,3,m-1608357555-1682084460,会议审议通过了关于强化企业科技创新主体地位的意见关于加强和改进国有经济管理有力支持中国式现代...,会议强调，加强和改进国有经济管理，要立足新时代新征程国有经济肩负的使命任务和功能定位，从服务...,企业发展
4,4,m-1608357555-1682084460,会议审议通过了关于强化企业科技创新主体地位的意见关于加强和改进国有经济管理有力支持中国式现代...,会议指出，支持民营经济发展是党中央的一贯方针。促进民营经济发展壮大，要着力优化民营经济发展环...,企业


In [13]:
df_para.shape

(3540, 5)

In [14]:
topics_pred = df_para['TOPIC'].unique().tolist()

In [16]:
titles = df_title['TITLE'].tolist()

In [19]:
topics_title = df_title['TITLE_TOPIC'].unique().tolist()

## Find the most similar topic from the pre-defined list

In [20]:
def find_most_similar_vectors(A, B, threshold = 0.7):
    most_similar_indices = []

    for vector_a in A:
        similarities = cosine_similarity([vector_a], B)
        if np.max(similarities) >= threshold:
          most_similar_index = np.argmax(similarities)
          most_similar_indices.append(most_similar_index)
        else:
          most_similar_indices.append(-1)

    return most_similar_indices

In [21]:
def get_most_similar_topic(topic_a, topic_b, threshold = 0.7):
  embeddings_a = [get_glm_embedding(str(x))[0][0] for x in topic_a]
  embeddings_b = [get_glm_embedding(str(x))[0][0] for x in topic_b]
  similar_idx = find_most_similar_vectors(embeddings_a, embeddings_b, threshold)
  a_b = {}
  for i in range(len(topic_a)):
    a_b[topic_a[i]] = topic_b[similar_idx[i]]
  return a_b

In [22]:
map_pred = get_most_similar_topic(topics_pred, topics_pdf)

In [23]:
map_title = get_most_similar_topic(topics_title, topics_pdf)

In [25]:
map_title_topic = {}
for index, row in df_title.iterrows():
  map_title_topic[row['TITLE']] = map_title[row['TITLE_TOPIC']]

In [26]:
df_para['TOPIC_PRED'] = df_para['TOPIC'].map(map_pred)
df_para['TOPIC_TITLE'] = df_para['TITLE'].map(map_title_topic)

## Aggregate to document level

In [37]:
def group_topics(group):
  topics_group = group['TOPIC_PRED'].tolist()
  counter = Counter(topics_group)
  top = counter.most_common(2)
  topics_all = [group['TOPIC_TITLE'].iloc[0]]
  topics_all.extend([x[0] for x in top])
  topics_all = list(set(topics_all))
  return pd.DataFrame({'ID': group.name,'TOPICS_ALL':[' '.join(topics_all)]})

In [38]:
result = df_para.groupby('ID').apply(group_topics).reset_index(drop=True)

In [40]:
result.to_csv('/content/id_topic_2.csv', encoding='utf_8_sig', index = False)